In [86]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
import numpy as np
from io import BytesIO
from datetime import datetime, timedelta
import boto3
from src.config import *
from src.s3_utils import get_s3_object

pd.set_option('display.max_columns', None)

Pulling Raw Data

In [87]:
file_path = r"C:\Users\jklas\Downloads\eu_demand_results.csv"
file = pd.read_csv(file_path)

cvp_agl_path = r"C:\Users\jklas\Downloads\cvp_agl.csv"
cvp_agl_uk = pd.read_csv(cvp_agl_path)

cvp_amzl_uk= get_s3_object(scheduling_bucket, "UK_CVP_Plan_All")

au_sg_fcst_path = r"C:\Users\jklas\Downloads\au_sg_fcst.csv"
au_sg_fcst = pd.read_csv(au_sg_fcst_path)

In [88]:
# missing SSD fcsts --> where to find?
# what is "over-night" cycle in AU/SG?

Maestro File Processing

In [89]:
file = file.copy()

file = file.rename(columns={
    "forecast_date": "date"
}) 

relevant_cols = [
    "station",
    "business_type",
    "cycle",
    "date",
    "forecast_car_vol",
    "forecast_lpv_vol"
]

cycle_map = {
    "cy1": "CYCLE_1",
    "cy2": "CYCLE_2",
    "sd_a": "CYCLE_SD_A",
    "sd_b": "CYCLE_SD_B",
    "sd_c": "CYCLE_SD_C",
    "adhoc": "AD_HOC",
    "day": "day",
    "brunch": "brunch",
    "lunch": "lunch",
    "bedtime": "bedtime",
    "dinner": "dinner",
    "sd": "sd"
}

file["cycle"] = file["cycle"].map(cycle_map, na_action=None)
file = file[file["cycle"] != "sd"]                                          # TODO: Removing cycle name "sd" as all forecasts in the Maestro file are zero for this cycle
file = file[relevant_cols]
file["date"] = pd.to_datetime(file["date"], dayfirst=False)
file["forecast_sum"] = file["forecast_car_vol"] + file["forecast_lpv_vol"]
file["car_share"] = np.where(file["forecast_sum"] == 0, 0, file["forecast_car_vol"] / file["forecast_sum"])
file["lpv_share"] = np.where(file["forecast_sum"] == 0, 0, file["forecast_lpv_vol"] / file["forecast_sum"])

file.head()

,station,business_type,cycle,date,forecast_car_vol,forecast_lpv_vol,forecast_sum,car_share,lpv_share
0,CC06,GSF,day,2025-06-01,13.438086,13.561914,27.0,0.497707,0.502293
1,CC06,GSF,day,2025-06-02,12.442672,12.557328,25.0,0.497707,0.502293
2,CC06,GSF,day,2025-06-03,11.944965,12.055035,24.0,0.497707,0.502293
3,CC06,GSF,day,2025-06-04,12.442672,12.557328,25.0,0.497707,0.502293
4,CC06,GSF,day,2025-06-05,11.447258,11.552742,23.0,0.497707,0.502293


CVP AMZL Data Processing

In [90]:
cvp_amzl_uk = cvp_amzl_uk.copy()

cvp_amzl_uk = cvp_amzl_uk.rename(columns={
    "ofd_date": "date",
    "ofd_volume": "snop_forecast"
}) 
cvp_amzl_uk["business_type"] = "AMZL"

relevant_cols = [
    "station",
    "business_type",    
    "cycle",            
    "date",    
    "snop_forecast"       
]


cvp_amzl_uk["date"] = pd.to_datetime(cvp_amzl_uk["date"], dayfirst=False)
cvp_amzl_uk["snop_forecast"] = cvp_amzl_uk["snop_forecast"] * cvp_amzl_uk["flex_share"]
cvp_amzl_uk = cvp_amzl_uk[relevant_cols]

cvp_amzl_uk.head()

,station,business_type,cycle,date,snop_forecast
0,DEH1,AMZL,CYCLE_1,2025-08-22,2529.319876
1,DCE1,AMZL,CYCLE_SD_C,2025-05-31,930.000000
2,DWR1,AMZL,CYCLE_SD_C,2025-08-02,1346.792243
3,DXN1,AMZL,CYCLE_SD_D,2025-05-29,545.931327
4,DRR1,AMZL,CYCLE_1,2025-08-16,981.464454


CVP AGL Data Processing

In [91]:
cvp_agl_uk = cvp_agl_uk.copy()

cvp_agl_uk.columns

Index(['station', 'provider_type', 'business_type', 'ship_type', 'cycle',
       'ofd_date', 'ofd_week', 'forecastwk', 'forecast', 'delivered'],
      dtype='object')

In [92]:
cvp_agl_uk = cvp_agl_uk.copy()

cvp_agl_uk = cvp_agl_uk.rename(columns={
    "ofd_date": "date",
    "forecast": "snop_forecast"
}) 

relevant_cols = [
    "station",
    "business_type",       
    "cycle",            
    "date",    
    "snop_forecast"           
]

cvp_agl_uk = cvp_agl_uk[relevant_cols]
cvp_agl_uk["date"] = pd.to_datetime(cvp_agl_uk["date"], dayfirst=False)
cvp_agl_uk["business_type"] = "GSF"
cvp_agl_uk["cycle"] = "day"

cvp_agl_uk.head()

,station,business_type,cycle,date,snop_forecast
0,CMC2,GSF,day,2021-02-17,296.0
1,ULI2,GSF,day,2022-03-08,607.0
2,CU10,GSF,day,2021-10-03,605.0
3,CU27,GSF,day,2021-04-20,298.0
4,CU35,GSF,day,2022-08-20,177.0


CVP SSD Data Processing

In [93]:
# TBD TBD TBD TBD TBD

CVP AU/SG Data

In [94]:
au_sg_fcst = au_sg_fcst.copy()

au_sg_fcst = au_sg_fcst.rename(columns={
    "forecast_delivery_date": "date",
    "pickup_location": "station",
    "dispatch_cycle": "cycle",
    "forecast_volume": "snop_forecast",
}) 

cycle_map = {
    "Cycle1": "CYCLE_1",
    "Cycle2": "CYCLE_2",
    "Same_day_1": "CYCLE_SD_A",
    "Same_day_2": "CYCLE_SD_B",
    "Over_Night": "Over_Night"
}

# au_fcst["cycle"] = au_fcst["cycle"].map(cycle_map, na_action=None)
au_sg_fcst["date"] = pd.to_datetime(au_sg_fcst["date"], dayfirst=False)
au_sg_fcst = au_sg_fcst[au_sg_fcst["last_mile_carrier"]=="Flex"]

au_sg_fcst.head()

,file_creation_date,date,station,cycle,source_method_volume,snop_forecast,local_file_identifier,last_update_date,last_mile_carrier
0,2025-05-30 00:00:00,2025-01-01,DNS4,Cycle1,local,0.0,station_forecast_22_00,2025-05-30 00:00:00,Flex
1,2025-05-30 00:00:00,2025-01-02,DNS4,Cycle1,local,0.0,station_forecast_22_01,2025-05-30 00:00:00,Flex
2,2025-05-30 00:00:00,2025-01-03,DNS4,Cycle1,local,0.0,station_forecast_22_02,2025-05-30 00:00:00,Flex
3,2025-05-30 00:00:00,2025-01-04,DNS4,Cycle1,local,0.0,station_forecast_22_03,2025-05-30 00:00:00,Flex
4,2025-05-30 00:00:00,2025-01-05,DNS4,Cycle1,local,0.0,station_forecast_22_04,2025-05-30 00:00:00,Flex


Joining all forecasts 

In [95]:
# Stack the CVP + AGL forecasts

forecasts = pd.concat([
    cvp_amzl_uk,
    cvp_agl_uk
], ignore_index=True)

forecasts.head()



,station,business_type,cycle,date,snop_forecast
0,DEH1,AMZL,CYCLE_1,2025-08-22,2529.319876
1,DCE1,AMZL,CYCLE_SD_C,2025-05-31,930.000000
2,DWR1,AMZL,CYCLE_SD_C,2025-08-02,1346.792243
3,DXN1,AMZL,CYCLE_SD_D,2025-05-29,545.931327
4,DRR1,AMZL,CYCLE_1,2025-08-16,981.464454


Joining the Data

In [96]:
# Date Comparison

cvp_rng = forecasts["date"].sort_values(ascending=False).unique()
print(f"""
      CVP______
      Max: {cvp_rng[0]}
      Min: {cvp_rng[-1]}
      """)

maestro_rng = file["date"].sort_values(ascending=False).unique()
print(f"""
      Maestro______
      Max: {maestro_rng[0]}
      Min: {maestro_rng[-1]}
      """)


      CVP______
      Max: 2025-09-20 00:00:00
      Min: 2020-04-01 00:00:00
      

      Maestro______
      Max: 2025-11-29 00:00:00
      Min: 2025-06-01 00:00:00
      


In [97]:
# Join tables
join_cols = ["date", "station", "cycle", "business_type"]

df = file.merge(
    forecasts,
    on= join_cols,
    how="left",
    suffixes=["_maestro", "_snop"]
)

tolerance = 2
df["difference"] = df["forecast_sum"] - df["snop_forecast"]
df["difference"] = np.where(abs(df["difference"]) > tolerance, df["difference"], 0)

df["match"] = np.where((df["difference"] > 0), False, True)
df["match"] = np.where(df["snop_forecast"].isnull(), np.nan, df["match"])

filter = (df["date"] > datetime.now()) & (df["date"] < (datetime.now() + timedelta(weeks=20)))          #TODO: Constraining the view to 10 Weeks

df = df[filter]


df.head()

,station,business_type,cycle,date,forecast_car_vol,forecast_lpv_vol,forecast_sum,car_share,lpv_share,snop_forecast,difference,match
23,CC06,GSF,day,2025-06-24,12.442672,12.557328,25.0,0.497707,0.502293,NaN,0.0,NaN
24,CC06,GSF,day,2025-06-25,12.442672,12.557328,25.0,0.497707,0.502293,NaN,0.0,NaN
25,CC06,GSF,day,2025-06-26,12.940379,13.059621,26.0,0.497707,0.502293,NaN,0.0,NaN
26,CC06,GSF,day,2025-06-27,15.428913,15.571087,31.0,0.497707,0.502293,NaN,0.0,NaN
27,CC06,GSF,day,2025-06-28,16.424327,16.575673,33.0,0.497707,0.502293,NaN,0.0,NaN


Comparing the Data

In [98]:
differences = df[df["match"] == 0]
matches = df[df["match"] == 1]
no_val = df[df["match"].isnull()]

print(f"""
    Rows overall:
    {df.shape[0]} ({df.query('business_type == "SSD"').shape[0]} SSD)
    ---------------------
    Count of Differences:
    {differences.shape[0]} ({round(differences.shape[0]/df.shape[0]*100)}%) ({differences.query('business_type == "SSD"').shape[0]} SSD)
    ---------------------
    Count of Matches:
    {matches.shape[0]} ({matches.query('business_type == "SSD"').shape[0]} SSD)
    ---------------------
    NaN values:
    {no_val.shape[0]} ({no_val.query('business_type == "SSD"').shape[0]} SSD)
    """
)



    Rows overall:
    59080 (5180 SSD)
    ---------------------
    Count of Differences:
    3402 (6%) (0 SSD)
    ---------------------
    Count of Matches:
    7386 (0 SSD)
    ---------------------
    NaN values:
    48292 (5180 SSD)
    


In [99]:
pd.set_option('display.max_columns', None)

# Show every unique combination of "station" and "cycle" in differences
differences[["station", "cycle"]].drop_duplicates()

,station,cycle
2583,DNG1,CYCLE_1
2950,DNG2,CYCLE_1
3180,DNG2,CYCLE_SD_A
3300,DNG2,CYCLE_SD_C
3669,DNN1,CYCLE_1
...,...,...
74111,DLS2,CYCLE_SD_C
74473,DLS4,CYCLE_1
74827,DXS1,CYCLE_1
75007,DXS1,CYCLE_SD_A


In [100]:
differences["cycle"].unique()

array(['CYCLE_1', 'CYCLE_SD_A', 'CYCLE_SD_C', 'CYCLE_SD_B', 'CYCLE_2'],
      dtype=object)

In [102]:
df.to_csv(r'C:\Users\jklas\Downloads\maestro_forecast_comparison.csv', index=False)